In [1]:
import desc
import tensorflow as tf
from time import time                                                       
import sys

2023-10-09 11:47:01.868944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-09 11:47:01.868970: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [18]:
import scanpy as sc

In [57]:
adata = sc.read("/home/lugli/spuccio/Projects/SP039/GMB_revisionSept2023/H5ad/Part2_Treg_inputDESC.h5ad")

In [58]:
adata = adata.raw.to_adata()

In [59]:
adata

AnnData object with n_obs × n_vars = 1933 × 16482
    obs: 'SampleID', 'GEO_RNA', 'Cohort', 'Type', 'Subtype', 'Diagnosis', 'Grade', 'Location', 'IDH1_Status', 'EGFR_Status', 'CellCount', 'Sample', 'Sex', 'Age', 'Sorted', 'batch', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'louvain'
    var: 'Gene type', 'Gene description', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Cohort_colors', 'SampleID_colors', 'Type_colors', 'hvg', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    obsp: 'connectivities', 'distances'

In [61]:
adata = adata[:,~adata.var.index.isin(adata[:,adata.var_names.str.startswith('MT-').tolist()].var.index)]

In [62]:
adata = adata[:,~adata.var.index.isin(adata[:,adata.var_names.str.startswith('RPL').tolist()].var.index)]

In [63]:
adata = adata[:,~adata.var.index.isin(adata[:,adata.var_names.str.startswith('RPS').tolist()].var.index)]

In [64]:
adata.raw = adata

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [65]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.25,n_top_genes=2000)
# This saves the original set of genes 
adata.raw = adata

adata = adata[:,adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)

/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [66]:
save_dir="/home/lugli/spuccio/Projects/Tmp"
adata=desc.train(adata,
        dims=[adata.shape[1],64,32],
        tol=0.005,
        n_neighbors=10,
        batch_size=256,
        louvain_resolution=[0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.2],# not necessarily a list, you can only set one value, like, louvain_resolution=1.0
        save_dir=str(save_dir),
        do_tsne=False,
        learning_rate=200, # the parameter of tsne
        use_GPU=False,
        num_Cores=15, #for reproducible, only use 1 cpu
        num_Cores_tsne=4,
        save_encoder_weights=False,
        save_encoder_step=3,# save_encoder_weights is False, this parameter is not used
        use_ae_weights=False,
        do_umap=True)

Start to process resolution= 0.2
The number of cpu in your computer is 56
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
use_ae_weights=False, the program will rerun autoencoder
Pretraining the 1th layer...
learning rate = 0.1
Epoch 1/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8257
Epoch 2/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8202
Epoch 3/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8145
Epoch 4/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8091
Epoch 5/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8052
Epoch 6/50
8/8 [==============================] - 0s 4ms/step - loss: 0.8018
Epoch 7/50
8/8 [==============================] - 0s 4ms/step - loss: 0.7983
Epoch 8/50
8/8 [==============================] - 0s 4ms/step - loss: 0.7943
Epoch 9/50
8/8 [==========================

/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/network.py:362: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y0.cat.categories=range(0,len(y0.cat.categories))
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/desc.py:159: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y_pred.cat.categories=list(range(len(y_pred.unique())))


umap finished and added X_umap0.2  into the umap coordinates (adata.obsm)

Start to process resolution= 0.3
The number of cpu in your computer is 56
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Checking whether /home/lugli/spuccio/Projects/Tmp/ae_weights.h5  exists in the directory
Pretraining time is 0.006707429885864258
61/61 [==============================] - 0s 1ms/step
...number of clusters is unknown, Initialize cluster centroid using louvain method
The value of delta_label of current 1 th iteration is 0.04035178479048112 >= tol 0.005
Epoch 1/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0316
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0295
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0276
Epoch 5/5
8/8 [=================

/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/network.py:362: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y0.cat.categories=range(0,len(y0.cat.categories))
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/desc.py:159: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y_pred.cat.categories=list(range(len(y_pred.unique())))


umap finished and added X_umap0.3  into the umap coordinates (adata.obsm)

Start to process resolution= 0.4
The number of cpu in your computer is 56
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Checking whether /home/lugli/spuccio/Projects/Tmp/ae_weights.h5  exists in the directory
Pretraining time is 0.006774187088012695
61/61 [==============================] - 0s 1ms/step
...number of clusters is unknown, Initialize cluster centroid using louvain method
The value of delta_label of current 1 th iteration is 0.06259699948266943 >= tol 0.005
Epoch 1/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0395
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0378
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0353
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0331
Epoch 5/5
8/8 [=================

...

umap finished and added X_umap0.9  into the umap coordinates (adata.obsm)

Start to process resolution= 1.0
The number of cpu in your computer is 56
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Checking whether /home/lugli/spuccio/Projects/Tmp/ae_weights.h5  exists in the directory
Pretraining time is 0.006420612335205078
61/61 [==============================] - 0s 1ms/step
...number of clusters is unknown, Initialize cluster centroid using louvain method
The value of delta_label of current 1 th iteration is 0.1717537506466632 >= tol 0.005
Epoch 1/5
8/8 [==============================] - 0s 4ms/step - loss: 0.0426
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0414
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0395
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0378
Epoch 5/5
8/8 [==================

/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/network.py:362: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y0.cat.categories=range(0,len(y0.cat.categories))
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/desc.py:159: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y_pred.cat.categories=list(range(len(y_pred.unique())))


umap finished and added X_umap1.0  into the umap coordinates (adata.obsm)

Start to process resolution= 1.2
The number of cpu in your computer is 56
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Checking whether /home/lugli/spuccio/Projects/Tmp/ae_weights.h5  exists in the directory
Pretraining time is 0.006791830062866211
61/61 [==============================] - 0s 1ms/step
...number of clusters is unknown, Initialize cluster centroid using louvain method
The value of delta_label of current 1 th iteration is 0.19192964304190377 >= tol 0.005
Epoch 1/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0444
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0432
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0414
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.0397
Epoch 5/5
8/8 [=================

/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/network.py:362: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y0.cat.categories=range(0,len(y0.cat.categories))
/home/lugli/spuccio/anaconda3/envs/scanpy19/lib/python3.10/site-packages/desc/models/desc.py:159: FutureWarning: Setting categories in-place is deprecated and will raise in a future version. Use rename_categories instead.
  y_pred.cat.categories=list(range(len(y_pred.unique())))


umap finished and added X_umap1.2  into the umap coordinates (adata.obsm)

The run time for all resolution is: 107.26784181594849
After training, the information of adata is:
 AnnData object with n_obs × n_vars = 1933 × 2000
    obs: 'SampleID', 'GEO_RNA', 'Cohort', 'Type', 'Subtype', 'Diagnosis', 'Grade', 'Location', 'IDH1_Status', 'EGFR_Status', 'CellCount', 'Sample', 'Sex', 'Age', 'Sorted', 'batch', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'louvain', 'desc_0.2', 'desc_0.3', 'desc_0.4', 'desc_0.5', 'desc_0.6', 'desc_0.7', 'desc_0.8', 'desc_0.9', 'desc_1.0', 'desc_1.2'
    var: 'Gene type', 'Gene description', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Cohort_colors', 'SampleID_colors', 'Type_colors', 'hvg', 'louvain', 'louvain_colors', 'pca', 'umap', 'prob_matrix0.2', 'prob_matrix0.3', 'prob_matrix0.4', 'prob_matrix0.5', 'prob_matrix0.6', 'prob_matrix0.7', 'prob_matrix0.8', 'prob_matrix0.9', 'prob_matrix1.0', 'prob

In [67]:
adata = adata.raw.to_adata()

In [68]:
adata.write("/home/lugli/spuccio/Projects/SP039/GMB_revisionSept2023/H5ad/Part2_Treg_outputDESC.h5ad")